In [270]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm
import dash
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.svm import SVR
from dash.dependencies import Input, Output, State
from dash import dcc, html

Kaggle data

In [235]:
apps = pd.read_csv('appearances.csv')
player_val = pd.read_csv('player_valuations.csv')
players = pd.read_csv('players.csv')

csv = [apps, player_val, players]

Scrapped data

In [236]:
df_keep1 = pd.read_csv('scrapping/goalkeeping_17-18.csv')
df_keep2 = pd.read_csv('scrapping/goalkeeping_18-19.csv')
df_keep3 = pd.read_csv('scrapping/goalkeeping_19-20.csv')
df_keep4 = pd.read_csv('scrapping/goalkeeping_20-21.csv')
df_keep5 = pd.read_csv('scrapping/goalkeeping_21-22.csv')
df_keep6 = pd.read_csv('scrapping/goalkeeping_22-23.csv')
df_keep7 = pd.read_csv('scrapping/goalkeeping_23-24.csv')

df_def1 = pd.read_csv('scrapping/defense_17-18.csv')
df_def2 = pd.read_csv('scrapping/defense_18-19.csv')
df_def3 = pd.read_csv('scrapping/defense_19-20.csv')
df_def4 = pd.read_csv('scrapping/defense_20-21.csv')
df_def5 = pd.read_csv('scrapping/defense_21-22.csv')
df_def6 = pd.read_csv('scrapping/defense_22-23.csv')
df_def7 = pd.read_csv('scrapping/defense_23-24.csv')

df_pass1 = pd.read_csv('scrapping/passes_17-18.csv')
df_pass2 = pd.read_csv('scrapping/passes_18-19.csv')
df_pass3 = pd.read_csv('scrapping/passes_19-20.csv')
df_pass4 = pd.read_csv('scrapping/passes_20-21.csv')
df_pass5 = pd.read_csv('scrapping/passes_21-22.csv')
df_pass6 = pd.read_csv('scrapping/passes_22-23.csv')
df_pass7 = pd.read_csv('scrapping/passes_23-24.csv')

df_shoot1 = pd.read_csv('scrapping/shooting_17-18.csv')
df_shoot2 = pd.read_csv('scrapping/shooting_18-19.csv')
df_shoot3 = pd.read_csv('scrapping/shooting_19-20.csv')
df_shoot4 = pd.read_csv('scrapping/shooting_20-21.csv')
df_shoot5 = pd.read_csv('scrapping/shooting_21-22.csv')
df_shoot6 = pd.read_csv('scrapping/shooting_22-23.csv')
df_shoot7 = pd.read_csv('scrapping/shooting_23-24.csv')

df_possess1 = pd.read_csv('scrapping/possession_17-18.csv')
df_possess2 = pd.read_csv('scrapping/possession_18-19.csv')
df_possess3 = pd.read_csv('scrapping/possession_19-20.csv')
df_possess4 = pd.read_csv('scrapping/possession_20-21.csv')
df_possess5 = pd.read_csv('scrapping/possession_21-22.csv')
df_possess6 = pd.read_csv('scrapping/possession_22-23.csv')
df_possess7 = pd.read_csv('scrapping/possession_23-24.csv')

dataframes = [df_def1, df_def2, df_def3, df_def4, df_def5, df_def6, df_def7,
              df_shoot1, df_shoot2, df_shoot3, df_shoot4, df_shoot5, df_shoot6, df_shoot7, df_pass1, df_pass2, df_pass3, df_pass4, df_pass5, df_pass6,
              df_pass7, df_possess1, df_possess2, df_possess3, df_possess4, df_possess5, df_possess6, df_possess7]

, df_keep1, df_keep2, df_keep3, df_keep4, df_keep5, df_keep6, df_keep7

Dealing with Nans

In [237]:
for df in dataframes:
    if 'Unnamed: 0' in df.columns:
        df.drop(columns=['Unnamed: 0'], inplace=True)
    df.rename(columns = {'Player' : 'name'}, inplace=True)

    for col in df.columns:
        df[col] = df[col].fillna(0)
        

In [238]:
print(apps.isna().any())
apps = apps.dropna()
print(apps.isna().count())


appearance_id             False
game_id                   False
player_id                 False
player_club_id            False
player_current_club_id    False
date                      False
player_name                True
competition_id            False
yellow_cards              False
red_cards                 False
goals                     False
assists                   False
minutes_played            False
dtype: bool
appearance_id             1573073
game_id                   1573073
player_id                 1573073
player_club_id            1573073
player_current_club_id    1573073
date                      1573073
player_name               1573073
competition_id            1573073
yellow_cards              1573073
red_cards                 1573073
goals                     1573073
assists                   1573073
minutes_played            1573073
dtype: int64


In [239]:
for i in range(7):
    dataframes[i].rename(columns={"Def 3rd" : "Def 3rd tackle", "Mid 3rd" : "Mid 3rd tackle", "Att 3rd" : "Att 3rd tackle",
                                  "Att" : "Dribbles challenged"}, inplace=True)

print(df_def1.columns)
print(df_possess1.columns)
print(df_pass1.columns)
print(df_shoot1.columns)

Index(['name', 'Tkl', 'TklW', 'Def 3rd tackle', 'Mid 3rd tackle',
       'Att 3rd tackle', 'TklDribblers', 'Dribbles challenged', 'Tkl%', 'Lost',
       'Blocks', 'Sh', 'Pass', 'Int', 'Clr', 'Err'],
      dtype='object')
Index(['name', 'Touches', 'Def Pen', 'Def 3rd', 'Mid 3rd', 'Att 3rd',
       'Att Pen', 'Live', 'Att', 'Succ', 'Succ%', 'Tkld', 'Tkld%', 'Carries',
       'TotDist', 'PrgDist'],
      dtype='object')
Index(['name', 'passes completed', 'passes attempted', 'passes success rate',
       'total distance of passes (yards)', 'progressive distance',
       'short passes completed', 'short passes attempted',
       'short passes succes rate', 'medium passes completed',
       'medium passes attempted', 'medium passes succes rate',
       'long passes completed', 'long passes attempted',
       'long passes succes rate', 'assists', 'expected assists 2nd deegree',
       'expected assists', 'key passes', 'passes into 1/3',
       'passes into penalty area'],
      dtype='object'

Merging dataframes into seasons

In [240]:
i = 0
df_17_18 = dataframes[0]
df_18_19 = dataframes[1]
df_19_20 = dataframes[2]
df_20_21 = dataframes[3]
df_21_22 = dataframes[4]
df_22_23 = dataframes[5]
df_23_24 = dataframes[6]

for df in dataframes:
    if (i % 7 == 0) & (i >= 7):
        df_17_18 = pd.merge(df_17_18, df, on='name')
    elif (i % 7 == 1) & (i >= 7):
        df_18_19 = pd.merge(df_18_19, df, on='name')
    elif (i % 7 == 2) & (i >= 7):
        df_19_20 = pd.merge(df_19_20, df, on='name')
    elif (i % 7 == 3) & (i >= 7):
        df_20_21 = pd.merge(df_20_21, df, on='name')
    elif (i % 7 == 4) & (i >= 7):
        df_21_22 = pd.merge(df_21_22, df, on='name')
    elif (i % 7 == 5) & (i >= 7):
        df_22_23 = pd.merge(df_22_23, df, on='name')
    elif (i % 7 == 6) & (i >= 7):
        df_23_24 = pd.merge(df_23_24, df, on='name')
    i += 1


In [241]:
top5_leagues = [ 'FR1', 'GB1', 'ES1', 'IT1', 'L1']
top5_id = apps[apps['competition_id'].isin(top5_leagues)]['player_id']
top5_players = players[players['player_id'].isin(top5_id)]['player_id']
top5_players

0             10
1             26
2             65
3             77
4             80
          ...   
30477    1144999
30478    1146029
30484    1159022
30499    1176345
30502    1178474
Name: player_id, Length: 9149, dtype: int64

Adding yellow and red cards + minutes

In [242]:
start_date = '2012-08-31'
end_date = '2013-06-14'
apps['date'] = pd.to_datetime(apps['date'])

start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)
seasons_all_players = []

for x in range(12):
    season_filter = apps[(apps['date'] >= start_date) & (apps['date'] <= end_date) & (apps['player_id'].isin(top5_players))]
    yellow_cards_dict = season_filter.groupby('player_id')['yellow_cards'].sum().to_dict()
    red_cards_dict = season_filter.groupby('player_id')['red_cards'].sum().to_dict()
    minutes_played_dict = season_filter.groupby('player_id')['minutes_played'].sum().to_dict()

    df_yellow = pd.DataFrame(list(yellow_cards_dict.items()), columns=['player_id', 'yellow_cards'])
    df_red = pd.DataFrame(list(red_cards_dict.items()), columns=['player_id', 'red_cards'])
    df_minutes = pd.DataFrame(list(minutes_played_dict.items()), columns=['player_id', 'minutes_played'])

    season = pd.merge(df_yellow, df_red, on='player_id', how='outer')
    season = pd.merge(season, df_minutes, on='player_id', how='outer')

    seasons_all_players.append(season)

    start_date += pd.DateOffset(years=1)
    end_date += pd.DateOffset(years=1)

/var/folders/k0/npn1p10j29xbj8plvsqs20b80000gn/T/ipykernel_32367/3345916464.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apps['date'] = pd.to_datetime(apps['date'])


Merge with names

In [243]:
filt3 = player_val[player_val['player_id'].isin(top5_players.values)]
filt3_sorted = filt3.sort_values(by='player_id')

names_all_players = players[['name', 'player_id']][players['player_id'].isin(top5_players.values)]
merged_all_players = filt3_sorted.merge(names_all_players, on='player_id')

Add values

In [244]:
start_date = '2012-08-31'
end_date = '2013-06-14'
merged_all_players['date'] = pd.to_datetime(merged_all_players['date'])
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)
all_values = []

for i in range(12):

    name_val = merged_all_players[(merged_all_players['date'] >= start_date) & (merged_all_players['date'] <= end_date)]

    all_values.append(name_val)

    temp = all_values[i]['player_id'].drop_duplicates(keep='first')
    all_values[i]['player_id'] = temp
    all_values[i] = all_values[i].dropna()

    start_date += pd.DateOffset(years=1)
    end_date += pd.DateOffset(years=1)

/var/folders/k0/npn1p10j29xbj8plvsqs20b80000gn/T/ipykernel_32367/2284836244.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_values[i]['player_id'] = temp


In [245]:
for i in range(len(seasons_all_players)):
    seasons_all_players[i] = pd.merge(seasons_all_players[i], all_values[i], on='player_id', how='outer')
    seasons_all_players[i] = seasons_all_players[i].dropna()



In [246]:
encoder = LabelEncoder()

info_all = players[['player_id', 'date_of_birth', 'sub_position', 'position', 'height_in_cm']][players['player_id'].isin(top5_players.values)]
info_all['sub_position_encoded'] = encoder.fit_transform(info_all['sub_position'])
info_all['position_encoded'] = encoder.fit_transform(info_all['position'])

info_all.dropna(inplace=True)
info_all

,player_id,date_of_birth,sub_position,position,height_in_cm,sub_position_encoded,position_encoded
0,10,1978-06-09,Centre-Forward,Attack,184.0,3,0
1,26,1980-08-06,Goalkeeper,Goalkeeper,190.0,5,2
4,80,1981-03-18,Goalkeeper,Goalkeeper,194.0,5,2
7,132,1980-10-04,Attacking Midfield,Midfield,179.0,0,3
9,215,1981-08-16,Centre-Forward,Attack,193.0,3,0
...,...,...,...,...,...,...,...
30419,1111912,2004-05-28,Centre-Forward,Attack,191.0,3,0
30477,1144999,2005-03-24,Right-Back,Defender,178.0,11,1
30478,1146029,2004-05-13,Centre-Back,Defender,186.0,2,1
30484,1159022,2004-01-08,Centre-Back,Defender,190.0,2,1


In [247]:
year_counter = -12

for i in range(len(seasons_all_players)):
    seasons_all_players[i] = seasons_all_players[i].merge(info_all, on='player_id', how='outer')
    seasons_all_players[i] = seasons_all_players[i].dropna()
    seasons_all_players[i].rename(columns={'date_of_birth' : 'age'}, inplace=True)
    seasons_all_players[i]['age'] = (datetime.now().year - pd.to_datetime(seasons_all_players[i]['age']).dt.year) + year_counter
    year_counter += 1

In [248]:
for i in range(len(seasons_all_players)):
    seasons_all_players[i] = seasons_all_players[i].drop(columns=['date','current_club_id','player_club_domestic_competition_id','sub_position','position'])

In [250]:
df_17_18 = pd.merge(df_17_18, seasons_all_players[5], on='name')
df_18_19 = pd.merge(df_18_19, seasons_all_players[6], on='name')
df_19_20 = pd.merge(df_19_20, seasons_all_players[7], on='name')
df_20_21 = pd.merge(df_20_21, seasons_all_players[8], on='name')
df_21_22 = pd.merge(df_21_22, seasons_all_players[9], on='name')
df_22_23 = pd.merge(df_22_23, seasons_all_players[10], on='name')
df_23_24 = pd.merge(df_23_24, seasons_all_players[11], on='name')

In [259]:
season_stats_all_players = {
    'season_2017_2018': df_17_18,
    'season_2018_2019': df_18_19,
    'season_2019_2020': df_19_20,
    'season_2020_2021': df_20_21,
    'season_2021_2022': df_21_22,
    'season_2022_2023': df_22_23,
    'season_2023_2024': df_23_24
}

In [260]:
combined_stats_all = pd.concat(season_stats_all_players.values(), keys=season_stats_all_players.keys())
combined_stats_all

name   Tkl  TklW  Def 3rd tackle  \
season_2017_2018 0     Patrick van Aanholt  47.0  32.0            29.0   
                 1          Rolando Aarons   4.0   4.0             3.0   
                 2          Rolando Aarons   4.0   4.0             3.0   
                 3          Rolando Aarons   4.0   4.0             3.0   
                 4          Rolando Aarons   4.0   4.0             3.0   
...                                    ...   ...   ...             ...   
season_2023_2024 4045           Kurt Zouma  23.0  17.0            14.0   
                 4046        Igor Zubeldia  32.0  25.0            20.0   
                 4047     Martín Zubimendi  49.0  27.0            18.0   
                 4048     Szymon Żurkowski  15.0   6.0             5.0   
                 4049      Martin Ødegaard  49.0  17.0            18.0   

                       Mid 3rd tackle  Att 3rd tackle  TklDribblers  \
season_2017_2018 0               15.0             3.0          16.0   
                 1                1.0             0.0           4.0   
                 2                1.0             0.0           4.0   
                 3                1.0             0.0           4.0   
                 4                1.0             0.0           4.0   
...                               ...             ...           ...   
season_2023_2024 4045             9.0             0.0          10.0   
                 4046            10.0             2.0          19.0   
                 4047            29.0             2.0          23.0   
                 4048             8.0             2.0           5.0   
                 4049            21.0            10.0          17.0   

                       Dribbles challenged  Tkl%  Lost  ...  PrgDist  \
season_2017_2018 0                    34.0  47.1  18.0  ...   1966.0   
                 1                     6.0  66.7   2.0  ...    127.0   
                 2                     6.0  66.7   2.0  ...    352.0   
                 3                     6.0  66.7   2.0  ...    127.0   
                 4                     6.0  66.7   2.0  ...    352.0   
...                                    ...   ...   ...  ...      ...   
season_2023_2024 4045                 16.0  62.5   6.0  ...   2019.0   
                 4046                 26.0  73.1   7.0  ...   4725.0   
                 4047                 42.0  54.8  19.0  ...   2671.0   
                 4048                 15.0  33.3  10.0  ...    561.0   
                 4049                 58.0  29.3  41.0  ...   3514.0   

                       player_id  yellow_cards  red_cards  minutes_played  \
season_2017_2018 0       52119.0           6.0        0.0          2056.0   
                 1      258188.0           0.0        0.0           656.0   
                 2      258188.0           0.0        0.0           656.0   
                 3      258188.0           0.0        0.0           656.0   
                 4      258188.0           0.0        0.0           656.0   
...                          ...           ...        ...             ...   
season_2023_2024 4045   157509.0           4.0        0.0          2839.0   
                 4046   355628.0          14.0        1.0          3428.0   
                 4047   423440.0           4.0        0.0          3465.0   
                 4048   387234.0           3.0        0.0           853.0   
                 4049   316264.0           2.0        0.0          3430.0   

                       market_value_in_eur  age  height_in_cm  \
season_2017_2018 0               9000000.0   27         176.0   
                 1                750000.0   22         178.0   
                 2                750000.0   22         178.0   
                 3                750000.0   22         178.0   
                 4                750000.0   22         178.0   
...                                    ...  ...           ...   
season_2023_2024 4045           25000000.0   29    

Data preparation

In [280]:
X = combined_stats_all.drop(columns=["market_value_in_eur", 'name', 'player_id'])
y = combined_stats_all['market_value_in_eur']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=1)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.5, random_state=1)

Random Forest

In [275]:
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_valid)

mae = mean_absolute_error(y_valid, y_pred)
mse = mean_squared_error(y_valid, y_pred)
r2 = rf.score(X_valid, y_valid)
print(f'Sezon {2012+i}/{2013+i}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R-squared: {r2}')

Sezon 2023/2024
MAE: 4899412.402591306
MSE: 82629252842737.77
R-squared: 0.6549006879073369


Ridge Regression

In [276]:
param_grid = {'alpha': [0.1, 1, 10, 100, 1000]}
ridge = Ridge()
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)


best_model = grid_search.best_estimator_


y_pred = best_model.predict(X_valid)


mae = mean_absolute_error(y_valid, y_pred)
mse = mean_squared_error(y_valid, y_pred)
r2 = r2_score(y_valid, y_pred)

print(f'Best alpha: {grid_search.best_params_}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R-squared: {r2}')

Best alpha: {'alpha': 0.1}
MAE: 7313169.542753955
MSE: 129937195589439.14
R-squared: 0.45732007406011144


Linear Regression

In [277]:
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

lm = sm.OLS(y_train, X_train)
lm_fit = lm.fit()

y_pred = lm_fit.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')

lm_fit.summary()

Mean Absolute Error (MAE): 7462629.457548132


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     market_value_in_eur   R-squared:                       0.483
Model:                             OLS   Adj. R-squared:                  0.480
Method:                  Least Squares   F-statistic:                     143.3
Date:                 Fri, 12 Jul 2024   Prob (F-statistic):               0.00
Time:                         00:04:38   Log-Likelihood:            -1.8783e+05
No. Observations:                10651   AIC:                         3.758e+05
Df Residuals:                    10581   BIC:                         3.763e+05
Df Model:                           69                                         
Covariance Type:             nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.135e+07   1.07e+05    105.647      0.000    1.11e+07    1.16e+07
x1         -6.274e+18   8.49e+18     -0.739      0.460   -2.29e+19    1.04e+19
x2          -8.38e+05   5.66e+05     -1.481      0.139   -1.95e+06    2.71e+05
x3          3.335e+18   4.51e+18      0.739      0.460   -5.51e+18    1.22e+19
x4          2.738e+18    3.7e+18      0.739      0.460   -4.52e+18       1e+19
x5          9.535e+17   1.29e+18      0.739      0.460   -1.57e+18    3.48e+18
x6          5.569e+17   7.53e+17      0.739      0.460   -9.19e+17    2.03e+18
x7         -1.168e+18   1.58e+18     -0.739      0.460   -4.27e+18    1.93e+18
x8         -2.219e+05   1.32e+05     -1.678      0.093   -4.81e+05    3.72e+04
x9           6.96e+17   9.41e+17      0.739      0.460   -1.15e+18    2.54e+18
x10        -2.553e+18   3.45e+18     -0.739      0.460   -9.32e+18    4.22e+18
x11         1.138e+18   1.54e+18      0.739      0.460   -1.88e+18    4.16e+18
x12         1.913e+18   2.59e+18      0.739      0.460   -3.16e+18    6.98e+18
x13        -1.807e+05   2.59e+05     -0.697      0.486   -6.89e+05    3.28e+05
x14        -5.664e+05   2.96e+05     -1.914      0.056   -1.15e+06    1.36e+04
x15         5.611e+05   1.31e+05      4.269      0.000    3.03e+05    8.19e+05
x16         3.373e+06   4.24e+05      7.955      0.000    2.54e+06     4.2e+06
x17          1.46e+05   5.17e+05      0.282      0.778   -8.68e+05    1.16e+06
x18        -1.522e+06   5.93e+05     -2.566      0.010   -2.69e+06    -3.6e+05
x19        -4.055e+05   1.85e+05     -2.190      0.029   -7.69e+05   -4.26e+04
x20          6.61e+05   1.79e+05      3.689      0.000     3.1e+05    1.01e+06
x21         7.666e+05   1.79e+05      4.280      0.000    4.15e+05    1.12e+06
x22         1.332e+05   2.21e+05      0.604      0.546   -2.99e+05    5.66e+05
x23        -6.085e+05   2.15e+05     -2.834      0.005   -1.03e+06   -1.88e+05
x24        -2.261e+05    1.5e+05     -1.503      0.133   -5.21e+05    6.88e+04
x25         3.736e+05   1.44e+05      2.589      0.010    9.07e+04    6.56e+05
x26        -5.538e+05   4.01e+05     -1.380      0.167   -1.34e+06    2.33e+05
x27         3.154e+06    2.1e+06      1.500      0.134   -9.67e+05    7.27e+06
x28        -1.352e+07   8.85e+06     -1.528      0.127   -3.09e+07    3.82e+06
x29         1.151e+07    7.6e+06      1.514      0.130   -3.39e+06    2.64e+07
x30         5.037e+05   1.54e+05      3.263      0.001    2.01e+05    8.06e+05
x31         3.531e+07   1.14e+07      3.106      0.002     1.3e+07    5.76e+07
x32        -2.026e+07   9.34e+06     -2.168      0.030   -3.86e+07   -1.94e+06
x33         3.451e+05   1.92e+05      1.801      0.072   -3.05e+04    7.21e+05
x34        -1.032e+07   5.75e+06     -1.795      0.073   -2.16e+07    9.48e+05
x35        -2.805e+06    7.3e+05     -3.843      0.000   -4.24e+06   -

In [282]:
clf = LogisticRegression(random_state=0, penalty='l2', max_iter=1000).fit(X_train, y_train)
clf.predict(X_valid)
clf.predict_proba(X_valid)
clf.score(X_valid, y_valid)

0.14796732701154822

In [283]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 484us/step - loss: 370670005387264.0000 - val_loss: 397606463209472.0000
Epoch 2/100
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 332us/step - loss: 364305534943232.0000 - val_loss: 395940955422720.0000
Epoch 3/100
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 333us/step - loss: 351626086842368.0000 - val_loss: 391470062239744.0000
Epoch 4/100
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 336us/step - loss: 359721328443392.0000 - val_loss: 383549605675008.0000
Epoch 5/100
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 335us/step - loss: 336414721966080.0000 - val_loss: 371872361349120.0000
Epoch 6/100
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 338us/step - loss: 340709219500032.0000 - val_loss: 356819272728576.0000
Epoch 7/100
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 332us/step - loss: 315134970953728.0000 - val_loss: 339521258389504.0000
Epoch 8/100
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 331us/step - loss: 310878020829184.0000 - val_loss: 321067193204736.0000
Epoch 9/100
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 332us/step - loss: 2932217122652

In [306]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error





# Define the model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

# Train the model
history = model.fit(X_train, y_train, epochs=15000, batch_size=128, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = model.predict(X_valid)
mse = mean_absolute_error(y_valid, y_pred)
r2 = r2_score(y_valid, y_pred)
print(f'Mean Absolute Error: {mse}')
print(f'R2: {r2}')


Epoch 1/15000


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11272121.0000 - val_loss: 11581963.0000
Epoch 2/15000
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 575us/step - loss: 11370196.0000 - val_loss: 11581083.0000
Epoch 3/15000
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 565us/step - loss: 11506733.0000 - val_loss: 11578437.0000
Epoch 4/15000
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step - loss: 11256612.0000 - val_loss: 11572979.0000
Epoch 5/15000
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step - loss: 11410206.0000 - val_loss: 11563676.0000
Epoch 6/15000
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step - loss: 11417012.0000 - val_loss: 11549717.0000
Epoch 7/15000
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step - loss: 11330233.0000 - val_loss: 11530241.0000
Epoch 8/15000
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step - loss: 11216138.0000 - val_loss: 11504352.0000
Epoch 9/15000
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step - loss: 11417040.0000 - val_loss: 11471721.0000
Epoch 10/15000
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 616us/step - loss: 11309650.0000 - val_loss: